<a href="https://colab.research.google.com/github/NapsterZ4/saturdays_second_edition/blob/master/cataratas_detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Montaje desde google drive

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


# Importando librerías

In [ ]:
import tensorflow as tf
import os
import numpy as np
from sklearn.model_selection import train_test_split

from tensorflow.keras import layers, models
import matplotlib.pyplot as plt

# Cargando datasets

## Etiquetado y extrayendo información

In [ ]:
files = []
labels = []

for dirname, _, filenames in os.walk('/content/drive/My Drive/Colab Notebooks/cataratas_images'):
  for filename in filenames:
    if filename.endswith('.png'):
      files.append(os.path.join(dirname, filename))
      l = np.zeros(4)

      if filename.startswith('NL'):
        l[0] = 1
      
      elif filename.startswith('ca'):
        l[1] = 1

      elif filename.startswith('Gl'):
        l[2] = 1

      elif filename.startswith('Re'):
        l[3] = 1
      
      labels.append(1)

print(len(labels), len(files))

601 601


## Train Test

In [ ]:
combined = list(zip(files, labels))
np.random.shuffle(combined)
files[:], labels[:] = zip(*combined)
print("Files:{}, labels:{}".format(files, labels))

Files:['/content/drive/My Drive/Colab Notebooks/cataratas_images/2_glaucoma/Glaucoma_094.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/2_cataract/cataract_075.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/1_normal/NL_295.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/2_cataract/cataract_029.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/2_glaucoma/Glaucoma_009.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/1_normal/NL_102.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/2_cataract/cataract_031.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/2_glaucoma/Glaucoma_075.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/1_normal/NL_131.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/1_normal/NL_223.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images/2_glaucoma/Glaucoma_079.png', '/content/drive/My Drive/Colab Notebooks/cataratas_images

In [ ]:
import re

images = []
directories = []
dircount = []
prevRoot=''
cant=0
 
imgpath = "/content/drive/My Drive/Colab Notebooks/cataratas_images/"

for root, dirnames, filenames in os.walk(imgpath):
    for filename in filenames:
        if re.search("\.(jpg|jpeg|png|bmp|tiff)$", filename):
            cant=cant+1
            filepath = os.path.join(root, filename)
            image = plt.imread(filepath)
            images.append(image)
            b = "Leyendo..." + str(cant)
            print (b, end="\r")
            if prevRoot !=root:
                print(root, cant)
                prevRoot=root
                directories.append(root)
                dircount.append(cant)
                cant=0
dircount.append(cant)
 
dircount = dircount[1:]
dircount[0]=dircount[0]+1
print('Directorios leidos:',len(directories))
print("Imagenes en cada directorio", dircount)
print('suma Total de imagenes en subdirs:',sum(dircount))

/content/drive/My Drive/Colab Notebooks/cataratas_images/1_normal 1


# Nueva forma de cargar los datos

In [8]:
# Cargando librerias
from __future__ import absolute_import, division, print_function, unicode_literals
from tensorflow.keras import layers, models

import matplotlib.pylab as plt
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np

In [4]:
data_root = ''
print(data_root)

data_root = "/content/drive/My Drive/Colab Notebooks/cataratas_images"
print(data_root)


/content/drive/My Drive/Colab Notebooks/cataratas_images


In [5]:
# Utilizamos un generador de datos aleatorios para sacar las imagenes de prueba y entrenamiento

IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)
print(TRAINING_DATA_DIR)

datagen_kwargs = dict(rescale =1./255, validation_split=.20)
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)

valid_generator = valid_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    subset="validation",
    shuffle=True,
    target_size=IMAGE_SHAPE
)

train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(
    TRAINING_DATA_DIR,
    subset="training",
    shuffle=True,
    target_size=IMAGE_SHAPE
)

/content/drive/My Drive/Colab Notebooks/cataratas_images
Found 120 images belonging to 4 classes.
Found 481 images belonging to 4 classes.


In [6]:
# Train dataset
image_train, label_train = next(iter(train_generator))
print(image_train.shape)
print(label_train.shape)

(32, 224, 224, 3)
(32, 4)


# Construcción del modelo CNN

In [12]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu', input_shape=(32,32,3)))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D(2,2))
model.add(layers.Conv2D(64, (3,3), activation='relu'))

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          36928     
Total params: 56,320
Trainable params: 56,320
Non-trainable params: 0
_________________________________________________________________


In [21]:
model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(10, activation='softmax'))

In [23]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 4, 4, 64)          36928     
_________________________________________________________________
flatten (Flatten)            (None, 1024)              0         
_________________________________________________________________
dense (Dense)                (None, 64)               

## Compilación y entrenamiento del modelo

In [22]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

history = model.fit(image_train, label_train, epochs=10, validation_data=(valid_generator))

Epoch 1/10


ValueError: ignored